# Introduction to Ablator

Welcome to the first chapter, where we're going to dive deep into the backstory of Ablator. We'll take a closer look at two important topics: Ablation Studies and Hyperparameter Optimization and learn why Ablator plays a crucial role in these areas. We'll also get to know some important modules that Ablator offers and take a peek behind the scenes to see what happens when Ablator kicks into action. 

## Understanding Ablation Studies and Hyperparameter Optimization

### Ablation Studies

In the context of Machine learning (ML) research, an [ablation study](https://en.wikipedia.org/wiki/Ablation_(artificial_intelligence)) is an experimental analysis used to understand the impact of different components of a Machine Learning model. This involves removing specific parts of a neural network architecture or changing different aspects of the training process to examine their contributions to the model's performance.

In fact, the process of removing or modifying components of a model corresponds to the process of choosing different values of **hyperparameters**, which are parameters that define the model architecture or training process. For example, the number of layers in a neural network, the number of neurons in each layer, the learning rate, the batch size, the number of epochs, etc. are all hyperparameters.

By selectively removing or modifying components/parameters, aka varying the hyperparameters combination (which we also refer to as configuration), researchers observe how the changes affect the system's output, performance, or behavior. Typically, hyperparameters are randomly chosen from a set of possible values called search space, which can either be discrete or continuous.

### Hyperparameter Optimization

Given the above definition of hyperparameters, we know that they are not models' parameters which are learned from the training process, but rather set by the user before training begins. Different combinations of hyperparameters can significantly impact the performance and convergence of a machine learning model.

[Hyperparameter Optimization (HPO)](https://en.wikipedia.org/wiki/Hyperparameter_optimization) involves systematically searching for the best set of hyperparameters that lead to optimal model performance. The objective is to find the hyperparameter combination that helps the model reach the best metrics (e.g minimum loss value, or maximum accuracy) among all possible training attempts. This is different than ablation study, where combinations of hyperparameters are chosen in a way (most of the time random) that lets us understand the impact of different components of a machine learning model.

Similarly to ablation studies, users will also define a search space for each hyperparameter, which is the base for our "searching" of the best hyperparameters.

## Ablator's Role: Enhancing Ablation Studies and Hyperparameter Optimization

As machine learning models grow in complexity, the number of components that need to be ablated also increases. This consequently expands the search space of possible configurations, requiring an efficient approach to horizontally scale multiple parallel experimental trials. Ablator is a tool that aids in the horizontal scaling of experimental trials.

Instead of manually configuring and conducting multiple experiments with various hyperparameter settings, Ablator automates this process. It initializes experiments based on different hyperparameter configurations, tracks the state of each experiment, and provides experiment persistence. 

Ablator employs a random search algorithm when conducting ablation studies, and a greedy search strategy known as Tree-structured Parzen Estimators (TPE) for HPO. TPE approach aims to efficiently optimize the hyperparameters by iteratively selecting values that maximize performance, leading to improved overall results. These searching paradigms terminates when the user-defined total number of trials is reached.

A few advantages of using Ablator are:

- It is a tool that simplifies the process of prototyping and training machine learning models. It streamlines model experimentation and evaluation.

- It offers a flexible configuration system.

- It facilitates result interpretation through visualization.

## Inside Ablator: Exploring its Modules and How They Work Together

This section introduces the core modules within Ablator. We'll cover the Configuration Module, Training Module, Experiment Result Metrics Module, and Analysis Module. These modules collectively contribute to the framework's seamless experiment management and insightful analysis capabilities. 

### [Configuration Module](../config.rst)

In Ablator, the configuration system serves as a foundation for crafting experiments. It enables researchers to set up the fine details. These settings are neatly grouped into various categories for easy organization:

- Model configuration

- Training configuration

- Optimizer and Scheduler configuration

- Running configurations

Flexibility is a big plus with this system. It lets researchers tailor their experiments to their specific aims and hypotheses.

### [Training Module](../training.rst)

This module handles the execution of the experiment, whether it is a single prototype experiment (using `ProtoTrainer`) or a parallel experiment with ablation study/ HPO (using `ParallelTrainer`). It has a training interface, `ModelWrapper`, that encapsulates common boilerplate code to abstract the repetitive tasks. The Configuration Module and the Trainer Module are blueprints for shaping the experiment.

### [Experiment result metrics module](../results.rst)

This class plays a pivotal role in capturing the performance of models. It receives predictions and outputs after the training of the ML model, and applies specific evaluation functions to calculate metrics.

### [Analysis Module](../analysis.rst)

The Analysis module in Ablator provides essential tools for visualizing and interpreting the experiment outcomes. It consists of two main classes: `PlotAnalysis` and `Results`. This module comes into play after training and evaluation are completed. It takes the experiment results, which include metrics and hyperparameters, and processes them using the `Results` class. The processed data is then visualized using the `PlotAnalysis` class, creating insightful graphs.

## What happens in the background once Ablator is launched? 

<img src="./Images/introduction.png" alt="Ablator overview">

1. Ablator initializes a Ray cluster for parallel execution. As specified, trials are sampled randomly from the search space in ablation studies, and greedily via TPE algorithm in HPO experiments. In both cases, Optuna is employed to collect and possibly analyze the trials performance to effectively guide the exploration for the next trials within the search space.

2. Trials are scheduled to run in parallel on available resources in the Ray cluster. The experiment directory is synchronized after each trial is completed, regardless of success or failure. This ensures that trial-specific information and results are stored for analysis (**Scheduling** from the Figure).

3. Suppose the user-specified total number of trials is not reached (aka it's greater than the sum of the trials that have been completed and those that are currently pending). In that case, the system initiates new trials. New trials are passed to initiate additional parallel training on the ray cluster. These are scheduled on available resources in the Ray cluster, potentially replacing completed trails. This way, utilization of the available resources is maximized.

4. After all trials are completed, a final synchronization of the entire experiment directory is done. All trial data, metrics, logs, and other relevant information are transferred for further analysis and storage. From this result, additional post-processing, analysis, and result aggregation tasks can be performed by the users. Moreover, Ablator's Analysis module aids in crafting visual plots that help understand how different components affect the model's performance, going beyond just looking at individual trial results. (**Syncing** and **Analysis** from the Figure)

The utilization of the Ray cluster's parallel processing capabilities, coupled with the effective search space exploration, provides an efficient way to perform ablation studies and elevate machine learning model performance.

To give you an idea of how experiments with ablator are written, we provide you with [these illustrative examples](./GettingStarted-more-demos.ipynb).